# Crossover Episodes - Picard Has Sex in Space

## Sex and the City: The Next Generation More Episodes

## SATC:TNG

In [ ]:
import pandas as pd
import re

In [ ]:
satc = pd.read_csv("train.csv")

In [ ]:
tng = pd.read_csv("train_tng.csv")

In [ ]:
satc.head()

In [ ]:
tng.head()

In [ ]:
satc['text'] = satc['text'].replace(satc['label'] ,'PICARD', regex = True)

In [ ]:
tngsatc = pd.concat([tng,satc])

In [ ]:
len(satc)

In [ ]:
len(tng)

In [ ]:
len(tngsatc)

In [ ]:
tngsatc.head()

In [ ]:
tngsatc.tail()

In [ ]:
tngsatc = tngsatc.dropna(axis=0)

tngsatc['is_valid'] = False

tngsatc.to_csv('tngsatc.csv')

### Read in our Edited Data Frame

In [1]:
from pathlib import Path
import pandas as pd
import re
import os
from fastai.text import *
import string
import numpy as np

%reload_ext autoreload
%autoreload 2
%matplotlib inline

cwd = os.getcwd()
path = Path(cwd)

### Here is the factory method for implementing our own vocab and tokenizer methods if we choose to do so

In [ ]:
all_letters = list(string.printable + string.whitespace) #Letters need to be in a list not a string

vocab=Vocab.create(all_letters, max_vocab=80000, min_freq=0)
customtokenizer = Tokenizer(pre_rules= [], post_rules=[])

processors = [TokenizeProcessor(),
             NumericalizeProcessor()]

data = (TextList.from_csv(path, 'tngsatc.csv', cols=['text'])
                 .split_by_rand_pct(0.01)
                 .label_for_lm()
                 .databunch(bs=80))
data.save('data_block_lm_both.pkl')

In [2]:
data_lm = load_data(path, 'data_block_lm_both.pkl',bs=80)
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.4)

C:\Users\brent\AppData\Roaming\Python\Python37\site-packages\fastai\datasets.py:156: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  with open(fpath, 'r') as yaml_file: return yaml.load(yaml_file)


In [ ]:
data_lm
data_lm.show_batch()

In [ ]:
learn.lr_find()
learn.recorder.plot(skip_end=15)

In [ ]:
learn.fit_one_cycle(20, 1e-2, moms=(0.8,0.7))

In [ ]:
learn.save('fit_head_both')

In [3]:
learn.load('fit_head_both')

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (147876 items)
x: LMTextList
xxbos xxup description : : xxmaj the xxup u.s.s. xxmaj enterprise xxup ncc xxup xxunk traveling at warp speed through space .,xxbos xxup picard xxup v.o. : : xxmaj captain 's log , stardate xxunk . xxmaj our destination is planet xxmaj cygnus xxup iv , beyond which lies the great unexplored mass of the galaxy .,xxbos xxup description : : on the gigantic new xxmaj enterprise xxup ncc xxup 1701-d.,xxbos xxup picard xxup v.o. : : xxmaj my orders are to examine xxmaj farpoint , a starbase built there by the inhabitants of that world . xxmaj meanwhile ...,xxbos xxup description : : xxmaj huge , with a giant wall diagram showing the xxunk of this xxmaj galaxy xxmaj class starship .
y: LMLabelList
,,,,
Path: C:\Users\brent\Desktop\satc writer;

Valid: LabelList (1493 items)
x: LMTextList
xxbos xxup description : : xxmaj the group moves off in casual clusters . xxmaj troi finds herself face to face with xxmaj 

In [4]:
learn.unfreeze()

In [5]:
learn.fit_one_cycle(20, 1e-3, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,2.517663,2.809579,0.445119,02:21
1,2.499608,2.795373,0.449554,02:20
2,2.478955,2.780712,0.450804,02:20
3,2.446410,2.768216,0.454434,02:21
4,2.411715,2.758357,0.454345,02:20
5,2.366935,2.755721,0.456429,02:20
6,2.326389,2.757265,0.456845,02:12
7,2.265467,2.766526,0.458006,02:10
8,2.223644,2.784796,0.456964,02:10
9,2.167418,2.796038,0.456548,02:10


In [6]:
learn.save('fine_tuned_both')
learn.save_encoder('fine_tuned_enc_both')

In [7]:
learn.fit_one_cycle(20, 1e-4, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,1.877931,2.890916,0.453304,02:10
1,1.877124,2.888594,0.453482,02:10
2,1.878566,2.892873,0.453304,02:10
3,1.893585,2.892801,0.453006,02:10
4,1.885365,2.894161,0.453065,02:10
5,1.882745,2.895905,0.452202,02:10
6,1.874473,2.900966,0.452946,02:10
7,1.865440,2.903513,0.452202,02:10
8,1.866338,2.904595,0.451756,02:10
9,1.867082,2.905867,0.452351,02:10


In [8]:
learn.save('fine_tuned_both')
learn.save_encoder('fine_tuned_enc_both')

# We have our trained model Now

In [9]:
from pathlib import Path
import pandas as pd
import re
import os
from fastai.text import *
import string
import numpy as np

%reload_ext autoreload
%autoreload 2
%matplotlib inline

cwd = os.getcwd()
path = Path(cwd)

In [11]:
data_lm = load_data(path, 'data_block_lm_both.pkl',bs=80)
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)
learn.load('fine_tuned_both')
learn.load_encoder('fine_tuned_enc_both')

C:\Users\brent\AppData\Roaming\Python\Python37\site-packages\fastai\datasets.py:156: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  with open(fpath, 'r') as yaml_file: return yaml.load(yaml_file)


In [12]:
learn.predict("WORF", 200, temperature=1.0)

"WORF : : i am glad that you will ever learn to ignore me for a long time . xxbos PICARD : : Mister Data -- energizing and reprogramming i find you ... xxbos DESCRIPTION : : Riker hurries out quickly , removing a big bag of Chips Ahoy . xxbos PICARD : : can i get You a drink ? xxbos GEORDI : : It 's a simple , sometimes ; Holographic chips . xxbos PICARD : : i wanted to delivery you in ten minutes . xxbos DESCRIPTION : : Picard glances at the Transporter Technician with an elaborately carved PADD . xxbos DATA 'S COM VOICE : : They are hailing us , sir . xxbos PICARD : : Not to all . xxbos RIKER : : i 'm sorry , if you 'll keep him under decision . xxbos TROI : : Captain , Geordi will not even acknowledge your vast experience with human eye level ... but i find his own way to end this ... xxbos"

### We Create Some Functions that Will Help us Write 

In [13]:
def GetCharacterLines(character, numberOfLines = 1, temperatureChange = True, temperature = 1.0):
    N_WORDS = 150
    EPSILON = 0.1
    #We predict the next N_WORDS
    #Each iteration increases the temperature of the sentence.  We interpolate between 0 and 1.
    if temperatureChange:
        predictedLines = list(learn.predict(character.upper(), N_WORDS,
                                            temperature=EPSILON + (i/(numberOfLines-1))).split("xxbos")[0] for i in range(numberOfLines))
    else:
        predictedLines = list(learn.predict(character.upper(), N_WORDS,
                                            temperature=temperature).split("xxbos")[0] for i in range(numberOfLines))
    return predictedLines

In [14]:
def RecursiveLine(line = "", numberOfLines = 5, temperature = 1):
    """Accept a line and continuously feeds it into the model to build a block of the script."""
    N_WORDS = 150
    dialog = ""
    for i in range(numberOfLines):
        #Sometimes we don't see the xxbos token and get an exception
        try:
            line = learn.predict(line, N_WORDS, temperature=temperature).split("xxbos")[i] + "xxbos"
        except:
            line = line

        dialog = dialog + line
    
    dialog = dialog.split("xxbos")[:-1]
    return dialog

In [15]:
def ConversationLines(characterList, numberOfLines = 5, temperature = 1):
    """Accept a list of character names.  We reject any sentences that do not begin with these particular characters.
    The same character will not speak 2 times in a row."""
    N_WORDS = 100
    dialog = ""
    line = ""
    lastCharacter = str(None)
    
    characterList = [character.upper() for character in characterList]
    
    for i in range(numberOfLines):
        #Keep creating lines until we have one of the characters
        while True:
            line = ""
            line = learn.predict(line, N_WORDS, temperature=temperature).split("xxbos")[0] + " xxbos "
            #Who is the current speaker for the line?
            currentCharacter = line.split(" ")[1]
            if (any(character in line for character in characterList)) and (currentCharacter != lastCharacter):
                break
    
        dialog = dialog + line
        lastCharacter = line.split(" ")[1]
    
    dialog = dialog.split("xxbos")[:-1]
    dialog = [line.lstrip() for line in dialog]
    return dialog

In [16]:
def ConversationLinesRecursive(characterList, numberOfLines = 5, temperature = 1):
    """Accept a list of character names.  We reject any sentences that do not begin with these particular characters.
    The same character will not speak 2 times in a row.
    Continuously Feeds the past dialog in to generate the text"""
    N_WORDS = 100
    dialog = ""
    lastCharacter = str(None)
    
    #Put the character list into all uppercase
    characterList = [character.upper() for character in characterList]
    
    #We generate the number of lines
    for i in range(numberOfLines):
        
        #Keep creating lines until we have one of the characters
        while True:
            line = learn.predict(dialog, N_WORDS, temperature=temperature).split("xxbos")[-1] + " xxbos "
            #Who is the current speaker for the line?
            currentCharacter = line.split(" ")[1]
            if (any(character in line for character in characterList)) and (currentCharacter != lastCharacter):
                break
    
        dialog = dialog + line
        lastCharacter = line.split(" ")[1]
        
    dialog = dialog.split("xxbos")[:-1]
    dialog = [line.lstrip() for line in dialog]
    return dialog

### Some Examples of Lines we can write automatically

In [39]:
RecursiveLine("Picard",numberOfLines=10,temperature=0.6)

['Picard : : And i will not be able to tell you that . You have to say , " Please , come in . ',
 " PICARD : : If you wish , i 'll see you tomorrow . ",
 ' PICARD : : You have a very lovely life . ',
 " PICARD : : - It 's a cult . ",
 ' PICARD : : i \'m not a " Single Person . " ',
 " PICARD : : You 're a big girl , you know ? ",
 " PICARD : : And i 'll go to the ladies ' room . ",
 ' DESCRIPTION : : The Borg is a little surprised at this . ',
 ' DESCRIPTION : : Picard ENTERS . He \'s wearing his Starfleet uniform on his uniform , and it \'s very similar to the one seen in " The Offspring " . He looks over the shoulder of Riker',
 ' PICARD : : No . But the Enterprise has been prepared to be the Enterprise in a few minutes , Commander . We must transmit a signal to the']

In [40]:
characters = ["Picard", "Riker", "Description"]
ConversationLines(characters, numberOfLines=5, temperature= 0.6)

['PICARD : : She has a quiet day .  ',
 'DESCRIPTION : : Picard , Riker , Data , Geordi , Troi , Beverly , Worf , Beverly , Geordi , and Worf . The VIEWSCREEN displays a starfield .  ',
 "PICARD : : It 's a fashion show .  ",
 'DESCRIPTION : : Picard stares at him , then turns and EXITS .  ',
 "PICARD : : It 's a New York birthday party .  "]

In [42]:
characters = ["Picard", "Riker", "Description"]
ConversationLinesRecursive(characters, numberOfLines=5, temperature= 0.6)

['DESCRIPTION : : He ',
 'PICARD : : This is ',
 'DESCRIPTION : : Riker and Data are working on a solution to the problem . The three ',
 "PICARD : : You know what ? i 'm gon na go . ",
 'DESCRIPTION : ']

In [43]:
GetCharacterLines("", numberOfLines=10, temperatureChange=False, temperature=0.6)

[" DESCRIPTION : : The doors open and Wesley ENTERS . Wesley ENTERS , carrying a PADD . He 's not sure what to do next . ",
 ' DESCRIPTION : : Riker and Worf EXIT . ',
 " PICARD : : i do n't know what i am . ",
 ' DESCRIPTION : : She moves to a wall panel , switches it " TECH " , as : ',
 ' PICARD : : You were in my bedroom when you were 16 and i was in Paris with the man . ',
 ' DESCRIPTION : : Picard EXITS . ',
 ' DESCRIPTION : : He looks at her with puzzlement . He decides to take another step back . ',
 ' DESCRIPTION : : Picard and Riker exchange a look ... ',
 ' PICARD : : Are you sure ? ',
 ' DESCRIPTION : : Data is at Science One with Data . ']

In [44]:
GetCharacterLines("Picard", 10)

["PICARD : : You 're not going to be in the mood . ",
 "PICARD : : You 're not going to be here , Data . ",
 "PICARD : : You 're not going to get married . ",
 "PICARD : : It 's a big deal . ",
 "PICARD : : You 're still not ready . ",
 "PICARD : : Counselor . Look at this . It 's just a couple of things ... ",
 "PICARD : : We 're just going to be in the middle of a fight and we 'll never get out of here . ",
 'PICARD : : So the idea of warp drive suggests that we could be together . ',
 "PICARD : : Oh , i hope that 's the conversation spin - Go ahead . ",
 'PICARD : : Let me help you . ']

# Area to Generate Dialog

In [50]:
GetCharacterLines("DESCRIPTION", 10)

['DESCRIPTION : : Picard and Riker exchange a look . ',
 'DESCRIPTION : : Picard , Riker , Data , Worf , Ensign Felton . Geordi is at the aft science station . ',
 'DESCRIPTION : : The Enterprise is docked at the Starbase . ',
 'DESCRIPTION : : He seems to be making a difficult decision . ',
 'DESCRIPTION : : The Bridge crew reacts with shock at what Riker sees . ',
 'DESCRIPTION : : She looks up at him , smiles . ',
 "DESCRIPTION : : The Administrator / Lieutenant turns and looks at Picard . He knows it 's impossible . ",
 'DESCRIPTION : : Picard studies the Tamarian . ',
 'DESCRIPTION : : She sits down , and once again , Picard gives a hug to her between the two . She searches for the right moment to make a troubled mistake , and when she helps her to comfort her , she finally leans to him . She studies her face and admits over her . ',
 'DESCRIPTION : : Completely disrupted . Stabilizes . ']

In [54]:
characters = ["RIKER", "DATA", "WORF", "DESCRIPTION", "PICARD"]
ConversationLines(characters, numberOfLines=20, temperature= 0.6)

['PICARD : : Charlotte had a little time to figure this out .  ',
 "DESCRIPTION : : Worf is trying to get his attention . He 's trying to figure out what he 's doing .  ",
 "PICARD : : It 's not that fast .  ",
 'RIKER : : Yes , sir .  ',
 "PICARD : : Well , let 's just say i thought you might .  ",
 'DESCRIPTION : : Riker and Worf exchange a look .  ',
 'PICARD : : Do you know what this is about ? Why are you doing this ?  ',
 'DESCRIPTION : : He looks at her , assessing .  ',
 "PICARD : : I 'm not sure . It does n't matter how you feel .  ",
 'DESCRIPTION : : There is a tense beat as the two men eye each other .  ',
 "PICARD : : But you do n't have to do that .  ",
 'DESCRIPTION : : Picard turns to Wesley .  ',
 "PICARD : : So , she 's the one with the great boyfriend .  ",
 'DESCRIPTION : : a scan of the planet MATERIALIZES . There is no indication of any activity .  ',
 "PICARD : : i 'm not meeting my people , Captain .  ",
 "DESCRIPTION : : Data 's head snaps up at the tone of his

In [55]:
learn.predict("PICARD", 200, temperature=0.6)

"PICARD : : He 's the liar . xxbos PICARD : : i do n't know . xxbos DESCRIPTION : : Picard and Troi EXIT . xxbos PICARD : : No . That 's a good question . xxbos DESCRIPTION : : She turns back to the console . xxbos PICARD : : i have a date with Jack . xxbos PICARD : : The Professor 's ship has been destroyed . Our missions have been rendered ineffective by the fleet . The Cardassians have left the Neutral Zone ... and i have no hope of deciphering the program . xxbos DATA : : Jenna is dead . xxbos PICARD : : My God , it 's a big deal . xxbos PICARD : : You 're gon na do it . xxbos DESCRIPTION : : She looks up from his console . xxbos PICARD : : At least we can get them into the Observation Lounge . xxbos PICARD : : If you 're"

In [56]:
GetCharacterLines("PICARD", 40)

["PICARD : : You 're not going to be naked in this world . ",
 "PICARD : : You 're not going to be here for a while ? ",
 "PICARD : : It 's not like that . ",
 "PICARD : : You 're not going to be here . ",
 "PICARD : : That 's the thing about New York . ",
 "PICARD : : You 're not going to take the wrong place . ",
 "PICARD : : You 're not going to be in a relationship with a man . ",
 "PICARD : : It 's just a little weird i 've been thinking about this . ",
 "PICARD : : It 's a beautiful night . ",
 "PICARD : : You 're not going to get married . ",
 "PICARD : : You 're the one who 's gone . ",
 "PICARD : : He 's not the one . ",
 "PICARD : : You 're not really in the mood for a movie . ",
 "PICARD : : He 's not a killer . ",
 'PICARD : : Okay , Thanks . ',
 "PICARD : : You 're not going to be here in your life . ",
 'PICARD : : There was a lot of men in there . ',
 'PICARD : : The Bridge is yours . ',
 "PICARD : : i do n't want to see you . ",
 "PICARD : : Now , very well . Let 's do 